In [10]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks


In [11]:


DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
    DEV = True
if DEV:
    epochs = 2
else:
    epochs = 10


train_data_path = 'imgs/train'


"""
Parameters
"""
img_width, img_height = 150, 150
batch_size = 32
samples_per_epoch = 1000
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 10
lr = 0.0004

model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))


model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))



model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')




/Users/patil1695/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,..., padding="same")`
/Users/patil1695/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
/Users/patil1695/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


Found 22424 images belonging to 10 classes.


In [17]:
validation_data_path = "imgs/train"
validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 22424 images belonging to 10 classes.


In [18]:
log_dir = './tf-log/'
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
cbks = [tb_cb]


model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)


target_dir = './models/'
if not os.path.exists(target_dir):
    os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

/Users/patil1695/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=10, validation_data=<keras_pre..., callbacks=[<keras.ca..., validation_steps=300, steps_per_epoch=31)`
  if sys.path[0] == '':


Epoch 1/10
31/31 [==============================] - 303s 10s/step - loss: 2.2749 - acc: 0.1663 - val_loss: 2.2324 - val_acc: 0.2902
Epoch 2/10
31/31 [==============================] - 341s 11s/step - loss: 2.2040 - acc: 0.2137 - val_loss: 2.1139 - val_acc: 0.2424
Epoch 3/10
31/31 [==============================] - 276s 9s/step - loss: 2.0967 - acc: 0.2772 - val_loss: 1.9656 - val_acc: 0.2858
Epoch 4/10
31/31 [==============================] - 290s 9s/step - loss: 2.0168 - acc: 0.3004 - val_loss: 1.8094 - val_acc: 0.4799
Epoch 5/10
31/31 [==============================] - 291s 9s/step - loss: 1.9227 - acc: 0.3387 - val_loss: 1.5694 - val_acc: 0.5162
Epoch 6/10
31/31 [==============================] - 299s 10s/step - loss: 1.8032 - acc: 0.3760 - val_loss: 1.4737 - val_acc: 0.5085
Epoch 7/10
31/31 [==============================] - 279s 9s/step - loss: 1.7113 - acc: 0.4083 - val_loss: 1.4164 - val_acc: 0.5290
Epoch 8/10
31/31 [==============================] - 269s 9s/step - loss: 1.6103 

In [19]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model


img_width, img_height = 150, 150
model_path = './models/model.h5'
model_weights_path = './models/weights.h5'
model = load_model(model_path)
model.load_weights(model_weights_path)

def predict(file):
    x = load_img(file, target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    result = array[0]
    answer = np.argmax(result)
    if answer == 0:
        print("C0")
    elif answer == 1:
        print("C1")
    elif answer == 2:
        print("C2")
    elif answer == 3:
        print("C3")
    elif answer == 4:
        print("C4")
    elif answer == 5:
        print("C5")
    elif answer == 6:
        print("C6")
    elif answer == 7:
        print("C7")
    elif answer == 8:
        print("C8")
    elif answer == 9:
        print("C9")
    return answer


In [20]:
c0_t = 0
c0_f = 0
c1_t = 0
c1_f = 0
c2_t = 0
c2_f = 0
c3_t = 0
c3_f = 0
c4_t = 0
c4_f = 0
c5_t = 0
c5_f = 0
c6_t = 0
c6_f = 0
c7_t = 0
c7_f = 0
c8_t = 0
c8_f = 0
c9_t = 0
c9_f = 0

for i, ret in enumerate(os.walk('imgs/validate/test/')):
    for i, filename in enumerate(ret[2]):
        if filename.startswith("."):
            continue
        result = predict(ret[0] + '/' + filename)
        if result == 0:
            c0_t += 1
        else:
            c0_f += 1
        if result == 1:
            c1_t += 1
        else:
            c1_f += 1
        if result == 2:
            c2_t += 1
        else:
            c2_f += 1
        if result == 3:
            c3_t += 1
        else:
            c3_f += 1
        if result == 4:
            c4_t += 1
        else:
            c4_f += 1
        if result == 5:
            c5_t += 1
        else:
            c5_f += 1
        if result == 6:
            c6_t += 1
        else:
            c6_f += 1
        if result == 7:
            c7_t += 1
        else:
            c7_f += 1
        if result == 8:
            c8_t += 1
        else:
            c8_f += 1
        if result == 9:
            c9_t += 1
        else:
            c9_f += 1
        
"""
Check metrics
"""
print("True c0: ", c0_t)
print("False c0: ", c0_f)
print("True c1: ", c1_t)
print("False c1: ", c1_f)
print("True c2: ", c2_t)
print("False c2: ", c2_f)
print("True c3: ", c3_t)
print("False c3: ", c3_f)
print("True c4: ", c4_t)
print("False c4: ", c4_f)
print("True c5: ", c5_t)
print("False c5: ", c5_f)
print("True c6: ", c6_t)
print("False c6: ", c6_f)
print("True c7: ", c7_t)
print("False c7: ", c7_f)
print("True c8: ", c8_t)
print("False c8: ", c8_f)
print("True c9: ", c9_t)
print("False c9: ", c9_f)


C5
C5
C1
C7
C5
C3
C4
C5
C6
C5
C7
C0
C4
C5
C5
C4
C6
C2
C6
C4
C7
C5
C5
C5
C6
C1
C5
C0
C4
C9
C6
C5
C5
C0
C7
C0
C2
C4
C5
C5
C1
C0
C7
C0
C0
C5
C5
C5
C0
C0
C9
C5
C0
C4
C0
C5
C0
C0
C9
C1
C6
C5
C6
C0
C0
C0
C4
C0
C5
C5
C4
C0
C0
C9
C5
C4
C9
C5
C6
C2
C0
C0
C6
C5
C3
C0
C0
C7
C1
C0
C0
C0
C7
C5
C5
C7
C3
C5
C0
C4
C1
C6
C2
C7
C5
C5
C5
C5
C5
C5
C0
C7
C0
C6
C0
C5
C7
C5
C9
C7
C0
C0
C5
C0
C0
C0
C5
C0
C3
C4
C5
C0
C4
C7
C6
C5
C0
C0
C5
C7
C5
C4
C5
C9
C1
C2
C4
C2
C4
C0
C0
C7
C0
C2
C7
C5
C6
C0
C4
C0
C5
C0
C1
C7
C0
C4
C4
C0
C4
C4
C7
C2
C1
C5
C1
C5
C5
C5
C7
C5
C5
C0
C9
C0
C5
C6
C0
C7
C0
C5
C5
C4
C3
C5
C6
C0
C5
C5
C5
C0
C1
C7
C5
C7
C5
C5
C0
C4
C5
C5
C6
C5
C3
C4
C5
C7
C5
C6
C6
C0
C0
C5
C8
C5
C0
C5
C6
C5
C0
C1
C1
C5
C6
C6
C6
C1
C3
C0
C3
C0
C5
C5
C0
C5
C0
C0
C0
C2
C5
C3
C5
C0
C0
C1
C5
C1
C0
C4
C5
C3
C5
C6
C6
C4
C5
C5
C1
C0
C4
C0
C4
C2
C4
C5
C0
C5
C0
C0
C5
C0
C0
C5
C5
C7
C6
C0
C6
C7
C0
C1
C3
C7
C5
C6
C0
C9
C9
C7
C7
C7
C5
C0
C5
C0
C1
C6
C5
C5
C7
C5
C0
C4
C0
C5
C7
C6
C0
C4
C5
C5
C0
C8
C6
C6
C6
C0
C6
C8
C1
C5
C1
C1
C7
C

C5
C5
C0
C0
C5
C5
C3
C7
C0
C3
C5
C9
C6
C6
C0
C0
C5
C5
C5
C6
C6
C5
C5
C0
C5
C7
C6
C7
C5
C4
C3
C1
C7
C0
C6
C0
C9
C4
C0
C9
C0
C6
C0
C9
C0
C6
C6
C6
C0
C5
C6
C5
C7
C5
C5
C0
C4
C0
C0
C9
C4
C5
C5
C4
C5
C7
C5
C0
C0
C6
C0
C4
C4
C5
C4
C0
C5
C0
C9
C0
C7
C5
C1
C7
C5
C5
C0
C5
C6
C0
C5
C0
C5
C0
C7
C5
C7
C5
C0
C6
C5
C5
C5
C5
C6
C5
C1
C5
C5
C0
C5
C5
C0
C8
C2
C5
C5
C4
C1
C5
C5
C0
C5
C0
C6
C5
C7
C0
C0
C4
C5
C5
C7
C1
C7
C0
C6
C0
C6
C0
C0
C6
C8
C1
C5
C7
C4
C9
C3
C5
C0
C0
C5
C2
C9
C0
C9
C0
C0
C6
C5
C0
C7
C4
C5
C5
C4
C6
C0
C4
C0
C7
C7
C5
C0
C1
C4
C4
C5
C3
C3
C2
C0
C4
C0
C5
C0
C5
C0
C5
C3
C3
C5
C2
C0
C5
C6
C5
C0
C7
C4
C0
C0
C4
C0
C4
C2
C0
C5
C3
C5
C6
C7
C8
C0
C6
C5
C0
C1
C7
C4
C5
C6
C0
C6
C5
C7
C5
C9
C0
C0
C5
C5
C6
C5
C6
C5
C7
C5
C0
C3
C5
C5
C0
C5
C7
C6
C5
C5
C5
C4
C5
C5
C1
C1
C0
C3
C2
C5
C3
C5
C0
C0
C7
C2
C1
C5
C5
C5
C9
C6
C5
C6
C4
C3
C7
C0
C5
C0
C0
C6
C5
C1
C1
C3
C5
C9
C0
C0
C0
C0
C5
C1
C5
C0
C5
C6
C0
C4
C0
C1
C0
C1
C9
C7
C0
C6
C7
C0
C5
C2
C7
C0
C4
C5
C7
C0
C0
C5
C3
C1
C7
C9
C0
C6
C5
C5
C0
C5
C6
C5
C5
C5
C

C0
C7
C3
C0
C7
C0
C0
C5
C5
C9
C9
C7
C5
C4
C6
C5
C0
C0
C1
C8
C5
C5
C7
C5
C6
C6
C4
C3
C0
C3
C1
C8
C6
C5
C6
C5
C0
C0
C4
C5
C1
C5
C6
C6
C7
C1
C1
C0
C9
C0
C0
C3
C6
C8
C0
C0
C6
C4
C0
C9
C5
C5
C0
C5
C5
C0
C4
C0
C5
C0
C5
C0
C0
C0
C3
C5
C5
C3
C7
C0
C5
C5
C0
C4
C1
C5
C5
C7
C1
C5
C3
C7
C5
C6
C9
C5
C2
C4
C6
C5
C0
C5
C0
C5
C0
C1
C5
C0
C9
C5
C7
C5
C6
C6
C0
C4
C9
C0
C0
C5
C1
C0
C4
C1
C1
C9
C5
C5
C6
C1
C7
C1
C0
C7
C5
C6
C9
C4
C0
C0
C0
C5
C5
C7
C5
C5
C9
C5
C7
C2
C1
C5
C7
C3
C0
C3
C0
C6
C1
C3
C0
C5
C0
C2
C6
C7
C5
C5
C5
C5
C6
C5
C0
C4
C0
C6
C0
C5
C1
C4
C0
C7
C6
C0
C7
C7
C0
C5
C7
C3
C6
C6
C2
C0
C5
C0
C5
C4
C1
C5
C5
C3
C5
C1
C1
C0
C0
C1
C0
C0
C0
C9
C0
C4
C3
C0
C5
C5
C0
C5
C2
C4
C5
C0
C3
C7
C9
C5
C0
C6
C5
C0
C9
C6
C5
C2
C6
C5
C7
C7
C0
C5
C5
C5
C7
C6
C0
C5
C0
C7
C5
C5
C4
C5
C0
C0
C0
C0
C0
C0
C0
C4
C0
C5
C4
C0
C6
C5
C0
C6
C5
C7
C0
C1
C5
C5
C6
C0
C1
C9
C9
C4
C0
C0
C0
C0
C9
C0
C4
C0
C6
C5
C5
C5
C5
C5
C1
C0
C0
C5
C0
C6
C5
C4
C5
C9
C7
C9
C5
C3
C5
C6
C1
C4
C5
C0
C0
C4
C1
C0
C5
C4
C5
C5
C5
C7
C4
C7
C6
C1
C0
C9
C0
C

C1
C0
C0
C5
C6
C5
C5
C5
C5
C5
C7
C9
C0
C0
C7
C5
C6
C5
C2
C7
C0
C5
C4
C0
C0
C2
C1
C4
C4
C5
C9
C0
C7
C5
C5
C5
C1
C6
C0
C5
C5
C5
C1
C5
C5
C4
C0
C0
C0
C6
C7
C3
C4
C0
C4
C1
C5
C5
C0
C5
C0
C4
C1
C3
C0
C5
C1
C4
C9
C0
C0
C9
C5
C0
C4
C5
C5
C4
C0
C1
C2
C5
C5
C0
C1
C0
C0
C0
C9
C4
C5
C7
C6
C6
C5
C0
C1
C5
C5
C1
C7
C9
C4
C3
C0
C6
C5
C3
C5
C0
C7
C6
C0
C9
C0
C4
C3
C7
C5
C0
C5
C0
C0
C1
C4
C5
C4
C9
C1
C5
C5
C0
C0
C0
C0
C5
C5
C5
C7
C5
C6
C5
C0
C5
C9
C5
C0
C5
C0
C6
C3
C4
C4
C0
C0
C5
C1
C6
C0
C2
C0
C5
C7
C1
C5
C6
C1
C2
C1
C6
C0
C0
C4
C5
C0
C1
C7
C0
C4
C1
C9
C5
C0
C6
C6
C5
C5
C5
C7
C5
C1
C7
C1
C7
C4
C7
C1
C0
C5
C2
C5
C6
C5
C5
C7
C5
C0
C5
C6
C5
C0
C9
C4
C0
C0
C4
C1
C3
C0
C4
C5
C5
C5
C5
C4
C4
C9
C4
C4
C0
C5
C1
C7
C5
C0
C1
C7
C6
C2
C5
C6
C2
C5
C0
C2
C5
C0
C5
C5
C0
C5
C5
C0
C4
C5
C0
C5
C5
C5
C5
C0
C9
C5
C0
C5
C1
C7
C5
C5
C9
C5
C7
C5
C5
C9
C5
C0
C5
C5
C0
C4
C1
C1
C0
C5
C3
C5
C4
C7
C7
C3
C4
C0
C0
C5
C5
C5
C0
C6
C1
C2
C5
C0
C5
C4
C0
C5
C5
C0
C5
C6
C0
C6
C0
C5
C5
C1
C1
C0
C4
C0
C9
C5
C1
C1
C4
C0
C5
C5
C7
C0
C5
C5
C

C5
C6
C0
C0
C6
C5
C5
C5
C4
C0
C0
C4
C4
C5
C0
C0
C7
C7
C4
C0
C6
C3
C5
C1
C6
C5
C0
C4
C7
C5
C0
C6
C1
C9
C9
C8
C0
C6
C0
C4
C4
C5
C0
C5
C0
C5
C0
C0
C1
C6
C5
C0
C5
C4
C7
C5
C6
C5
C7
C3
C0
C6
C0
C9
C5
C1
C7
C1
C5
C1
C0
C5
C7
C0
C2
C4
C9
C4
C0
C4
C5
C7
C0
C8
C1
C9
C1
C6
C5
C5
C9
C4
C5
C5
C7
C6
C0
C0
C5
C7
C5
C5
C5
C0
C0
C5
C5
C7
C0
C4
C7
C9
C5
C5
C1
C9
C5
C7
C0
C1
C0
C6
C7
C5
C4
C5
C1
C7
C5
C6
C7
C7
C1
C1
C0
C1
C0
C5
C6
C0
C5
C1
C5
C5
C9
C5
C4
C0
C5
C5
C0
C5
C5
C5
C5
C7
C6
C0
C2
C1
C0
C9
C0
C0
C5
C0
C0
C5
C0
C7
C3
C6
C0
C9
C2
C4
C6
C5
C5
C3
C0
C5
C3
C5
C5
C9
C0
C0
C1
C9
C5
C3
C0
C2
C9
C0
C5
C5
C5
C5
C6
C7
C0
C5
C0
C5
C1
C4
C5
C0
C6
C3
C9
C1
C5
C0
C9
C0
C6
C5
C0
C5
C4
C0
C0
C3
C0
C0
C5
C5
C5
C0
C0
C3
C4
C6
C0
C4
C4
C7
C4
C5
C4
C0
C4
C5
C5
C1
C4
C5
C7
C5
C0
C7
C3
C1
C0
C1
C5
C7
C7
C4
C2
C8
C7
C5
C7
C7
C0
C0
C4
C9
C5
C5
C4
C7
C1
C5
C5
C2
C7
C7
C1
C5
C1
C0
C0
C6
C0
C7
C2
C0
C4
C6
C0
C0
C7
C1
C4
C6
C0
C4
C5
C3
C3
C0
C1
C6
C5
C4
C6
C9
C0
C0
C5
C0
C0
C0
C7
C6
C5
C0
C0
C6
C7
C9
C5
C0
C5
C5
C0
C6
C4
C

...

C5
C0
C5
C0
C5
C6
C5
C0
C4
C5
C0
C4
C5
C9
C7
C0
C5
C3
C0
C7
C5
C4
C1
C4
C5
C9
C0
C0
C0
C5
C0
C1
C9
C4
C1
C1
C5
C6
C5
C5
C1
C9
C0
C0
C0
C0
C9
C0
C5
C0
C0
C0
C5
C5
C0
C5
C6
C7
C0
C4
C0
C0
C0
C0
C4
C7
C6
C9
C1
C1
C0
C0
C7
C7
C4
C0
C4
C5
C6
C5
C0
C0
C0
C0
C5
C5
C0
C5
C2
C0
C0
C4
C1
C6
C5
C6
C1
C5
C1
C5
C6
C1
C4
C5
C0
C7
C5
C5
C1
C0
C4
C0
C5
C7
C5
C1
C3
C0
C4
C5
C0
C0
C5
C6
C4
C5
C0
C9
C6
C5
C5
C5
C5
C5
C7
C9
C5
C5
C5
C6
C1
C5
C1
C5
C5
C5
C7
C1
C0
C5
C1
C0
C1
C5
C1
C0
C6
C6
C5
C7
C5
C4
C1
C0
C6
C4
C5
C4
C5
C7
C5
C1
C5
C5
C9
C5
C1
C3
C5
C0
C5
C5
C0
C7
C4
C5
C0
C3
C5
C4
C6
C5
C0
C5
C5
C0
C0
C7
C9
C7
C5
C1
C4
C0
C5
C5
C1
C6
C0
C5
C4
C0
C5
C5
C5
C0
C1
C6
C5
C1
C9
C0
C5
C6
C7
C4
C5
C6
C5
C0
C5
C0
C7
C4
C6
C7
C5
C6
C4
C5
C6
C7
C2
C5
C0
C7
C5
C0
C1
C3
C0
C5
C5
C5
C0
C0
C0
C7
C5
C6
C5
C0
C7
C4
C1
C4
C4
C0
C0
C6
C2
C6
C5
C7
C2
C5
C4
C4
C1
C7
C7
C0
C9
C4
C5
C0
C7
C5
C1
C1
C0
C0
C9
C0
C0
C5
C1
C5
C0
C5
C5
C0
C5
C4
C8
C1
C0
C1
C1
C5
C2
C9
C0
C2
C8
C7
C5
C0
C0
C5
C5
C0
C4
C5
C1
C0
C1
C5
C5
C1
C6
C7
C9
C

C1
C6
C5
C1
C0
C0
C6
C2
C0
C0
C1
C6
C4
C4
C0
C0
C1
C0
C6
C0
C6
C4
C5
C6
C3
C1
C3
C4
C5
C1
C5
C5
C7
C3
C5
C5
C7
C0
C7
C9
C5
C7
C9
C7
C5
C5
C5
C9
C6
C5
C5
C7
C3
C6
C5
C1
C4
C7
C5
C5
C6
C4
C6
C4
C7
C9
C6
C6
C5
C5
C6
C0
C5
C1
C5
C1
C4
C1
C5
C4
C5
C5
C0
C7
C5
C5
C6
C5
C0
C0
C0
C7
C6
C5
C0
C5
C5
C4
C0
C5
C5
C1
C5
C6
C0
C6
C4
C5
C0
C5
C0
C5
C5
C0
C8
C9
C0
C0
C0
C0
C0
C5
C7
C0
C7
C3
C7
C3
C0
C1
C0
C4
C0
C6
C0
C6
C5
C7
C4
C5
C0
C5
C0
C5
C0
C5
C1
C1
C6
C6
C5
C7
C6
C9
C0
C0
C0
C4
C6
C9
C7
C6
C5
C0
C0
C5
C6
C0
C5
C4
C9
C5
C4
C7
C6
C5
C5
C7
C0
C5
C6
C5
C7
C0
C7
C5
C0
C5
C5
C6
C0
C0
C5
C5
C4
C9
C5
C5
C4
C0
C9
C1
C5
C3
C0
C5
C6
C7
C2
C4
C9
C2
C0
C5
C9
C0
C1
C5
C1
C1
C0
C6
C5
C9
C2
C7
C5
C7
C1
C4
C6
C5
C7
C1
C5
C5
C7
C5
C5
C3
C0
C5
C5
C0
C5
C3
C5
C0
C5
C7
C5
C5
C1
C1
C5
C0
C4
C9
C9
C6
C0
C0
C6
C0
C0
C5
C0
C6
C0
C0
C5
C1
C6
C7
C7
C5
C4
C7
C3
C9
C1
C0
C7
C0
C9
C0
C5
C0
C5
C7
C5
C6
C5
C5
C5
C6
C5
C7
C6
C0
C1
C5
C5
C0
C4
C4
C0
C0
C0
C2
C5
C0
C5
C1
C0
C7
C0
C1
C9
C5
C5
C0
C4
C4
C5
C0
C4
C3
C6
C5
C1
C3
C5
C

C3
C5
C0
C9
C0
C6
C5
C5
C5
C0
C5
C1
C0
C8
C0
C6
C0
C5
C0
C0
C2
C1
C5
C0
C4
C4
C7
C1
C1
C0
C1
C0
C7
C0
C4
C0
C5
C0
C3
C6
C5
C3
C9
C5
C0
C0
C2
C3
C0
C0
C0
C4
C5
C5
C7
C0
C0
C0
C0
C6
C5
C0
C7
C0
C6
C5
C2
C4
C0
C0
C5
C9
C5
C4
C5
C7
C4
C0
C0
C6
C0
C6
C5
C6
C4
C2
C7
C2
C0
C0
C0
C0
C4
C5
C7
C9
C0
C1
C0
C3
C0
C4
C1
C2
C5
C6
C7
C5
C0
C1
C7
C5
C6
C1
C4
C4
C7
C5
C0
C5
C0
C7
C0
C4
C6
C0
C5
C5
C4
C6
C0
C7
C0
C6
C5
C5
C0
C6
C1
C4
C6
C2
C5
C5
C5
C6
C0
C5
C5
C5
C6
C0
C5
C5
C0
C5
C2
C6
C0
C4
C7
C5
C5
C6
C5
C0
C4
C0
C5
C5
C0
C7
C0
C5
C5
C5
C6
C5
C1
C5
C5
C0
C1
C6
C6
C5
C5
C5
C5
C1
C0
C5
C5
C4
C5
C7
C1
C4
C6
C5
C5
C6
C6
C5
C0
C0
C6
C4
C1
C0
C5
C0
C4
C5
C0
C5
C2
C0
C0
C9
C5
C7
C5
C5
C7
C4
C5
C0
C1
C0
C5
C1
C5
C2
C5
C7
C9
C9
C5
C7
C0
C5
C5
C5
C9
C4
C5
C5
C5
C5
C9
C0
C5
C5
C0
C5
C9
C0
C5
C0
C0
C4
C9
C5
C9
C5
C4
C7
C7
C6
C5
C4
C5
C6
C9
C0
C1
C3
C4
C6
C5
C4
C5
C7
C1
C0
C5
C4
C4
C6
C6
C4
C1
C7
C5
C7
C0
C5
C4
C5
C0
C5
C5
C2
C9
C0
C4
C0
C5
C5
C5
C7
C1
C5
C5
C1
C0
C2
C4
C2
C0
C5
C5
C0
C1
C5
C3
C7
C5
C0
C6
C5
C0
C

C9
C7
C5
C0
C7
C1
C2
C4
C9
C6
C5
C0
C5
C5
C1
C1
C0
C5
C7
C0
C3
C2
C5
C7
C0
C0
C5
C0
C6
C7
C0
C0
C5
C9
C0
C1
C5
C5
C6
C0
C5
C0
C6
C5
C5
C9
C5
C5
C5
C0
C4
C5
C4
C9
C6
C1
C0
C0
C7
C6
C0
C6
C5
C0
C6
C5
C5
C5
C0
C4
C0
C5
C1
C6
C6
C0
C5
C0
C7
C4
C5
C0
C0
C0
C2
C7
C7
C5
C7
C0
C0
C1
C5
C3
C0
C4
C5
C6
C6
C5
C0
C5
C0
C0
C6
C5
C0
C6
C1
C5
C5
C5
C0
C0
C1
C0
C4
C5
C0
C0
C6
C2
C7
C5
C0
C5
C0
C4
C5
C0
C7
C1
C5
C0
C6
C5
C5
C7
C0
C5
C5
C1
C5
C0
C6
C0
C0
C5
C5
C6
C4
C0
C0
C0
C5
C0
C5
C5
C5
C0
C5
C1
C7
C9
C5
C5
C3
C5
C2
C5
C1
C5
C6
C5
C9
C5
C0
C7
C1
C0
C1
C4
C9
C5
C5
C0
C2
C3
C5
C4
C3
C6
C0
C0
C0
C0
C5
C4
C0
C5
C9
C4
C0
C4
C5
C9
C0
C5
C1
C4
C0
C6
C6
C2
C0
C1
C7
C5
C3
C5
C7
C1
C7
C4
C9
C4
C0
C5
C5
C6
C5
C4
C5
C6
C0
C0
C7
C7
C5
C3
C6
C8
C1
C7
C1
C0
C2
C3
C7
C0
C5
C5
C1
C0
C6
C5
C5
C0
C0
C5
C6
C3
C1
C4
C0
C6
C0
C0
C7
C3
C0
C0
C0
C4
C5
C0
C9
C5
C6
C7
C0
C5
C5
C0
C0
C0
C7
C5
C4
C5
C5
C6
C5
C5
C1
C0
C0
C5
C5
C1
C4
C0
C5
C6
C0
C0
C6
C5
C0
C3
C7
C5
C0
C5
C9
C0
C4
C0
C5
C1
C5
C7
C5
C0
C0
C4
C1
C5
C4
C6
C5
C6
C9
C

C0
C6
C0
C5
C3
C6
C5
C5
C5
C7
C5
C0
C5
C5
C2
C0
C0
C9
C0
C2
C9
C5
C1
C5
C0
C6
C0
C5
C0
C0
C0
C6
C1
C5
C4
C5
C9
C0
C5
C0
C5
C0
C0
C0
C5
C4
C3
C0
C2
C6
C9
C6
C4
C1
C5
C0
C5
C0
C5
C1
C6
C7
C4
C1
C0
C3
C5
C5
C5
C2
C0
C5
C6
C4
C5
C9
C5
C0
C0
C5
C0
C5
C0
C0
C5
C5
C3
C0
C0
C5
C7
C2
C7
C9
C6
C7
C0
C0
C4
C5
C7
C2
C0
C1
C0
C4
C3
C5
C5
C7
C5
C2
C9
C4
C1
C0
C9
C9
C4
C5
C0
C0
C5
C9
C0
C6
C1
C1
C0
C7
C4
C0
C6
C4
C7
C6
C5
C5
C0
C4
C9
C0
C3
C9
C1
C7
C0
C1
C6
C1
C0
C5
C6
C5
C1
C3
C5
C4
C5
C7
C5
C4
C0
C9
C0
C1
C6
C7
C5
C0
C5
C3
C7
C9
C5
C6
C5
C3
C5
C0
C5
C0
C5
C3
C5
C4
C7
C4
C2
C5
C5
C7
C0
C8
C7
C6
C9
C2
C5
C4
C7
C5
C5
C0
C5
C0
C0
C4
C3
C5
C5
C4
C5
C4
C1
C5
C5
C0
C0
C0
C1
C0
C5
C5
C0
C5
C6
C5
C0
C0
C0
C6
C6
C5
C6
C4
C6
C0
C1
C0
C5
C7
C4
C6
C0
C0
C0
C5
C5
C0
C7
C7
C2
C0
C9
C1
C5
C0
C5
C6
C3
C5
C9
C2
C5
C9
C5
C5
C9
C2
C2
C0
C5
C0
C5
C5
C4
C0
C5
C6
C7
C0
C0
C1
C0
C5
C5
C1
C7
C7
C0
C5
C5
C0
C6
C5
C0
C0
C4
C0
C6
C7
C4
C4
C0
C7
C0
C0
C7
C6
C4
C3
C9
C9
C0
C6
C2
C6
C4
C5
C0
C5
C0
C5
C0
C5
C5
C0
C0
C4
C0
C5
C0
C